In [ ]:
import boto3
from sagemaker import get_execution_role
from zipfile import ZipFile
import os 

# Get the SageMaker execution role
# sagemaker need permission to write lambda functions

# get the execution role for this notebook
#role = get_execution_role()

# Create a Lambda client using Boto3
client = boto3.client('lambda')



In [ ]:
role

In [ ]:
# Function to create Lambda function from a Python file
def create_lambda_function(function_name, file_name, role_arn, timeout = 10):
    """
    """
    # Zip the lambda_function.py file
    with ZipFile(f'{file_name}.zip', 'w') as z:
        z.write(f'{file_name}.py')
        
    # check that .zip file was created
    assert os.path.exists(f'{file_name}.zip'), f" File not created {file_name}.zip"
    
    # Read the zipped file content
    with open(f'{file_name}.zip', 'rb') as f:
        b_code = f.read()
        
    # Create the Lambda function
    response = client.create_function(
        FunctionName=function_name,
        Runtime='python3.8',
        Handler=f'{file_name}.lambda_handler',  # lambda handler entry point
        Code={'ZipFile': b_code},
        Description=f'{function_name} for image classification workflow',
        Timeout=timeout,
        MemorySize=1024,
        Publish=True,
        PackageType='Zip',
        # role with full access to lambda (write functions included)
        # also S3 full access
        Role="arn:aws:iam::676719617777:role/lambda_full_access"
    )
    
    print(f'Created Lambda function: {function_name}')
    return response


In [ ]:


# Create 3 Lambda functions
create_lambda_function('serializeImageData', 'serialize_image_data', role, 10 )
# give more time to the classification image
create_lambda_function('classifyImage', 'classify_image', role, 60)
create_lambda_function('filterInferences', 'filter_inferences', role, 10)


### Test Lambda Function 
- call the functions with a payload, a json file